In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import requests


# =================================================0
#                    基本設定
# ========================================2022.01.13
# Selenium Serverに接続
options = webdriver.ChromeOptions()
driver = webdriver.Remote(
    command_executor='scraping_selenium_1:4444/wd/hub',
    options=options,
)
# 要素が見つかるまで最大10秒待機
driver.implicitly_wait(10)


# ------------------------------
# 検索情報定義
url = 'https://qiita.com'
keyword = 'NoCode アプリ開発'
article_per_page = 10
records = [["タイトル"]]


# ------------------------------
# アクセス & 検索バーの操作
driver.get(url)
elm_serch_bar = driver.find_element(By.NAME, 'q')
elm_serch_bar.send_keys(keyword)
elm_serch_bar.submit()

# ------------------------------
# 記事数からループ回数を設定
elm_article_total_nums = driver.find_element(By.XPATH, '//li[@class="active"]/a/span')
article_total_nums = int(elm_article_total_nums.text)
roop_times = article_total_nums // article_per_page

# ------------------------------
# スクレイピング実行
for i in range(roop_times + 1):
    for article_title_h1 in driver.find_elements(By.XPATH, '//h1'):
        # CSVには二次元配列で渡す
        record = []
        record.append(article_title_h1.text)
        records.append(record)
        
    # 最終ページの場合は次へを押さずに終了
    if i == roop_times:
        break

    # 次のページへ遷移
    next_a = driver.find_element(By.CLASS_NAME, 'js-next-page-link')
    next_a.click()

# ------------------------------
# CSV出力
with open('./csv/scrauto.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(records)

driver.quit()

MaxRetryError: HTTPConnectionPool(host='scraping_selenium_1', port=4444): Max retries exceeded with url: /wd/hub/session (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdac4af6580>: Failed to establish a new connection: [Errno -2] Name or service not known'))